# Using a carbon budget

This tutorial will show how to use a carbon budget to limit carbon emissions.

Again, we will build off the default model that comes with MUSE. To copy the files for this model, run:

```bash
python -m muse --model default --copy PATH/TO/COPY/THE/MODEL/TO
```

## Adding a carbon budget

As the simulation will take much longer to run, will will first adjust the time framework to run only until 2035:

```toml
time_framework = [2020, 2025, 2030, 2035]
```

Then we will add the following to impose a carbon budget:

```toml
[carbon_budget_control]
budget = [300, 300, 300, 300]
commodities = ['CO2f']
method = 'bisection'
control_undershoot = false
control_overshoot = false
method_options.sample_size = 5
method_options.tolerance = 0.2
```

We need a value for each year in the time framework. In this case we are imposing a budget of 300 in every year. The other lines indicate settings used in the carbon budget algorithm, which modifies the carbon price so that carbon emissions come as close as possible to the budget.

Finally, since we want to look at the commodity supply to monitor carbon emissions, we will add the supply output, which will create a file `MCASupply.csv` with supply values for each commodity in each timeslice/year:

```toml
[[outputs]]
quantity = "supply"
sink = "aggregate"
filename = "{cwd}/{default_output_dir}/MCA{Quantity}.csv"
```

Now we can run the model with the usual command:

    python -m muse settings.toml

Once the model has run, we can import the necessary libraries and visualise the results. We will first look at the `MCASupply.csv` file to see carbon emissions. We will also look at the `MCAPrices.csv` file to see how the carbon price has been changed.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))

# CO2 emissions
df = pd.read_csv(
    "../tutorial-code/8-carbon-budget/1-carbon-budget/Results/MCASupply.csv"
)
df_sum = (
    df[["commodity", "year", "supply"]]
    .groupby(["commodity", "year"])
    .sum()
    .reset_index()
)
sns.barplot(data=df_sum[df_sum.commodity == "CO2f"], x="year", y="supply", ax=ax1)
carbon_profile = [300, 300, 300, 300]
ax1.plot(carbon_profile, c="r")
ax1.set_ylabel("CO2 emissions")
ax1.set_xlabel("Year")
ax1.tick_params(axis="x", labelrotation=90)

# CO2 prices
df = pd.read_csv(
    "../tutorial-code/8-carbon-budget/1-carbon-budget/Results/MCAPrices.csv"
)
sns.lineplot(
    data=df[df.commodity == "CO2f"], x="year", y="prices", hue="timeslice", ax=ax2
)
ax2.set_ylabel("CO2 price")
ax2.set_xlabel("Year")
ax2.set_ylim(bottom=0)
ax2.get_legend().remove()

fig.subplots_adjust(wspace=0.5)

We can see a substantial reduction in CO2 emissions from 2020 to 2025, driven by a sharp increase in the carbon price which is necessary to bring emissions within budget (shown by the red line).

Next, we will look at the `MCACapacity.csv` file, to see the investment decisions behind this outcome.

In [ ]:
mca_capacity = pd.read_csv(
    "../tutorial-code/8-carbon-budget/1-carbon-budget/Results/MCACapacity.csv"
)

fig, axes = plt.subplots(1, 3)
all_years = mca_capacity["year"].unique()
for ax, (sector_name, sector_data) in zip(axes, mca_capacity.groupby("sector")):
    sector_capacity = sector_data.groupby(["year", "technology"]).sum().reset_index()
    sector_capacity.pivot(
        index="year", columns="technology", values="capacity"
    ).reindex(all_years).plot(kind="bar", stacked=True, ax=ax)
    ax.set_ylabel("Capacity (PJ)")
    ax.set_xlabel("Year")
    ax.set_title(f"{sector_name.capitalize()} Sector:", fontsize=10)
    ax.legend(title=None, prop={"size": 8})
    ax.tick_params(axis="both", labelsize=8)

fig.set_size_inches(8, 2.5)
fig.subplots_adjust(wspace=0.5)

Compared to the default example, we can see substantial investment in `windturbine` in the power sector early on in the simulation. This can easily be explained by the increase in carbon price, which drives up the price of gas-driven energy technologies and makes renewables comparatively more attractive for investment.